### Experiment #5 (Hyperparameter Tuning)
- Script: mamdani_tsk_selection.py
- Raw data generation command:
```bash
	$ bin/it2tsk_hyperparameter_tuning
```
- Raw data result: IT2TSK_Hyperparameter_Tuning.xlsx
- Time required: total +/- 12 minutes

In [1]:
from notebook_resolver import *
import pandas as pd
from src.utils.excel_operation import export_to_excel

file_path = "experiments/IT2TSK_Hyperparameter_Tuning.xlsx"

# Load data hasil Optuna (berisi banyak run per sheet)
df = pd.read_excel(file_path)

# Ekstrak urutan numerik dari nama sheet (contoh: "01_BeefCuts" -> 1)
df["Sheet_Order"] = df["Sheet Name"].str.extract(r"^(\d+)").astype(int)

# Hitung statistik R² untuk setiap sheet
r2_stats = (
    df.groupby("Sheet Name")["R2"]
    .agg(
        R2_min="min",
        R2_max="max",
        R2_mean="mean",
        R2_median="median",
        R2_std="std",
    )
    .reset_index()
)

# Ambil baris terbaik (R² tertinggi) per sheet
df_best = df.loc[df.groupby("Sheet Name")["R2"].idxmax()].reset_index(drop=True)

# Gabungkan hasil terbaik dengan statistik R²
final_df = pd.merge(df_best, r2_stats, on="Sheet Name", how="left")

# Tambahkan urutan sheet agar bisa diurutkan dengan benar
final_df["Sheet_Order"] = final_df["Sheet Name"].str.extract(r"^(\d+)").astype(int)
final_df = final_df.sort_values("Sheet_Order").reset_index(drop=True)

# Pilih kolom yang ingin ditampilkan (sesuaikan sesuai kebutuhan)
final_df = final_df[
    [
        "Sheet Name",
        "R2",          # best R²
        "R2_min",
        "R2_max",
        "R2_mean",
        "R2_median",
        "R2_std",
        "Rule Base Clustering Time",
        "Features Clustering Time",
        "Clustering Time",
        "batch_size",
        "tol",
        "max_no_improvement",
        "uncertainty_factor",
        "min_std_ratio",
    ]
]

# Bulatkan angka agar rapi
cols_to_round = [
    "R2", "R2_min", "R2_max", "R2_mean", "R2_median", "R2_std",
    "Rule Base Clustering Time", "Features Clustering Time", "Clustering Time"
]
final_df[cols_to_round] = final_df[cols_to_round].round(4)

# Tambahkan baris Average di bagian bawah
avg_row = pd.DataFrame({
    "Sheet Name": ["Average (All Sheets)"],
    "R2": [final_df["R2"].mean()],
    "R2_min": [final_df["R2_min"].mean()],
    "R2_max": [final_df["R2_max"].mean()],
    "R2_mean": [final_df["R2_mean"].mean()],
    "R2_median": [final_df["R2_median"].mean()],
    "R2_std": [final_df["R2_std"].mean()],
    "Rule Base Clustering Time": [final_df["Rule Base Clustering Time"].mean()],
    "Features Clustering Time": [final_df["Features Clustering Time"].mean()],
    "Clustering Time": [final_df["Clustering Time"].mean()],
    "batch_size": [""],
    "tol": [""],
    "max_no_improvement": [""],
    "uncertainty_factor": [""],
    "min_std_ratio": [""],
}).round(4)

# Gabungkan hasil akhir + average row
final_df = pd.concat([final_df, avg_row], ignore_index=True)

# Simpan hasil ke Excel (sheet baru)
export_to_excel(final_df, file_path, "best_params_summary")

final_df


,Sheet Name,R2,R2_min,R2_max,R2_mean,R2_median,R2_std,Rule Base Clustering Time,Features Clustering Time,Clustering Time,batch_size,tol,max_no_improvement,uncertainty_factor,min_std_ratio
0,1.Inside-Outside,0.9797,0.9776,0.9797,0.9793,0.9795,0.0004,0.1552,1.8582,2.0134,512,0.00001,5,0.01,0.09
1,2.Round,0.9920,0.9871,0.9920,0.9915,0.9916,0.0008,0.1501,1.6894,1.8394,512,0.0001,5,0.01,0.06
2,3.Top_Sirloin,0.9811,0.9809,0.9811,0.9810,0.9810,0.0000,0.1580,1.7963,1.9542,1024,0.00001,5,0.01,0.06
3,4.Tenderloin,0.9922,0.9899,0.9922,0.9916,0.9918,0.0006,0.1456,1.7321,1.8777,512,0.001,5,0.01,0.15
4,5.Flap_meat,0.9861,0.9857,0.9861,0.9860,0.9860,0.0001,0.1610,1.6821,1.8432,256,0.00001,20,0.13,0.04
5,6.Striploin,0.9905,0.9885,0.9905,0.9901,0.9902,0.0004,0.1346,1.5841,1.7188,256,0.0001,5,0.18,0.09
6,7.Rib_eye,0.9923,0.9879,0.9923,0.9913,0.9921,0.0012,0.1347,1.5725,1.7072,256,0.0001,10,0.03,0.04
7,8.Skirt_meat,0.9948,0.9928,0.9948,0.9944,0.9947,0.0006,0.1318,1.5725,1.7042,256,0.001,5,0.07,0.06
8,9.Brisket,0.9933,0.9923,0.9933,0.9929,0.9930,0.0003,0.1590,1.7263,1.8853,512,0.0001,10,0.07,0.06
9,10.Clod_Chuck,0.9780,0.8584,0.9780,0.9628,0.9654,0.0201,0.1741,1.7388,1.9128,256,0.0001,20,0.01,0.13
